In [1]:
import os
import sys
import math
import csv
import numpy as np
import pandas as pd
import bct.algorithms
import scipy
from scipy.io import loadmat
from scipy import stats
from tabulate import tabulate
import dataframe_image as dfi
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
import random
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from ripser import ripser
from persim import plot_diagrams
from sklearn.utils import resample
from scipy import stats
import pickle
%matplotlib inline

In [2]:
#Reading the fMRI Data in the Matriux Format

In [3]:
path = "/home/sumita/Downloads/Research/NetNeuro/ADNI_fMRI_Frederick_Jiong/fmri-data-fullnetworks/"
subfolders = os.listdir(path)
subfolders_cleaned = []
# remove unnecessary folders
for item in subfolders:
    if not item.startswith('.') and not item.startswith('s'):
        subfolders_cleaned.append(item)
# access func_nets folder for each patient
subj_nets = pd.DataFrame(columns=['Subj_ID', 'Scale1_Net'])

for item in subfolders_cleaned:
    subpath = path + item + "/func_nets/"
    files = os.listdir(subpath)
    if "corr_atlas-L2018_res-scale1.csv" in files:
        
        with open(subpath + "corr_atlas-L2018_res-scale1.csv") as csvfile:
            
            mat=pd.read_csv(csvfile, sep=',',header=None)
            scale1_sub_net= np.array(mat)
        #print(np.shape(scale1_sub_net))
            
        if np.shape(scale1_sub_net) == (99, 99):
        
            new_row = {'Subj_ID': item, 'Scale1_Net':scale1_sub_net}
            subj_nets = subj_nets.append(new_row, ignore_index=True)


In [6]:
subj_mats_temp = subj_nets.to_numpy()

In [7]:
subj_mats = subj_mats_temp

In [8]:
for i in range(subj_mats_temp.shape[0]):
    subj_mats[i][1] = clean_arr(subj_mats_temp[i][1])
    

In [1]:
subj_demo = pd.read_csv("subject_demographics.csv")
subj_demo.head()

In [11]:
sub_id = np.array(subj_demo.PTID)
subj_diag = np.array(subj_demo.DX_bl)
subj_gender = np.array(subj_demo.PTGENDER)

In [2]:
#subj_demo.loc[subj_demo['PTGENDER'] == 'Male']
#subj_demo.loc[subj_demo['PTGENDER'] == 'Female']

In [13]:
# Efficiency requires a distance matrix of weights between 0 and 1
def inverse_RBF_old(W, with_inf=True, norm=False):
    W_new = np.zeros(W.shape)
    # Build a mask
    valid_edges = np.nonzero(W)
    mask = np.zeros(np.shape(W)).astype(bool)
    mask[valid_edges] = True
    zero_edges = np.argwhere(W==0)
    
    # Reverse RBF kernel
    # Denominator has a small margin to make sure that ln does not reach 1
    W_new[mask] = np.sqrt(-1 * np.log( W[mask]/(np.max(W[mask])+1e-5) ))
    
    # If we need to normalize between 0 and 1
    if norm == True:
        W_new = W_new/np.max(W_new[W_new<np.inf])
    
    # Fill in the non-edges as infinite distance
    if with_inf == True:
        W_new[~mask] = np.inf
    
    # Ensure that connections to self are 0 distance
    np.fill_diagonal(W_new, 0)
    
    return W_new

In [14]:
# Efficiency requires a distance matrix of weights between 0 and 1
def inverse_RBF(W):
    W_new = np.zeros(W.shape)
    (m,n) = np.shape(W)
    
    for i in range(0,m):
        for j in range(0,n):
            W_new[i][j] = 1-W[i][j]
    
    
    # Ensure that connections to self are 0 distance
    np.fill_diagonal(W_new, 0)
    
    return W_new

In [15]:
def PH_entropy(ripser_diagram, dim):
    s_D = 0
    E_D = 0
    d_M = np.max(clean_arr(np.array(ripser_diagram[dim][:,1])))

    for i in range(len(ripser_diagram[dim])):
        
        b_i = ripser_diagram[dim][i][0]
        d_i = ripser_diagram[dim][i][1] if ripser_diagram[dim][i][1] != math.inf else d_M

        p_i = d_i - b_i if d_i >= b_i else d_M
        s_D += p_i
    
    for i in range(len(ripser_diagram[dim])):
        b_i = ripser_diagram[dim][i][0]
        d_i = ripser_diagram[dim][i][1] if ripser_diagram[dim][i][1] != math.inf else d_M
        p_i = d_i - b_i if d_i >= b_i else d_M
        s_i = p_i/s_D
        if s_i <0: 
            print(b_i)
            print(d_i)
            print(s_i)
            print(d_M)
        E_D += s_i*(np.log(s_i)) # negative of entropy

    return -E_D

In [72]:
#for computing cocycles on N nodes

# Inputs: 
#     nets: ((M x M) X N) vector of N networks sized (M x M)
# Outputs:
#   Vertex Importance Profile
def compute_cocycles(subj_nets):
    
    #warnings.filterwarnings('ignore')
    np.set_printoptions(precision=12)
    np.set_printoptions(formatter={'float_kind':'{:f}'.format})
    
    
    #Avergae Persistence and Persistence Entropy
    #Global_0D_features_AP_PE =[]
    Global_1D_features_AP_PE =[]
    Global_1D_features_AP_PE_df =['Average Persistence', 'Persistence Entropy']

    #max_entropy_0D = 0
    #min_entropy_0D = np.inf
    
    max_entropy_1D = 0
    min_entropy_1D = np.inf


     
    #Vertex Importance Profile    
    
    # N = No of vertices/Nodes
    
    N = np.shape(subj_nets[0][1])[0]
    
    
    #Number of Subjects/People
    
    Num_of_Subjs = subj_nets.shape[0]
    
    Vertex_Importance =list(range(0, N))
    Vertex_Importance_NumCoCyles =list(range(0, N))
    Vertex_Importance_NumCoCyles.insert(N, -1) # Last elemnt on the list holds the total number of cocycles for each subject 
    Sub_ID_Vertex_Importance_NumCoCycles =  ['Sub_ID'] + Vertex_Importance_NumCoCyles # first elemnt on the list holds the sub id
    
    #for i in tqdm(range(0,10)):
    for i in tqdm(range(0,Num_of_Subjs)):
    
        #print("Subj ID: ", subj_nets[i][0])
        net= subj_nets[i][1]
        #print(net)

        
        
        #using RBF kernel
        
        sub = inverse_RBF(net)
        #print(sub)
       
    
        
        
        
        #Using Ripser to extract persistence barcodes
        ripser_diagram = ripser(net, distance_matrix=True)['dgms']
        
        #print(ripser_diagram)
        
        #Compute Average Persistence and Persistence Entropy
        
        
        #PH0 Features
        #birth_thresh_0D = clean_arr(np.array(ripser_diagram[0][:,0]))
        #death_thresh_0D = clean_arr(np.array(ripser_diagram[0][:,1]))
        #life_0D = np.subtract(death_thresh_0D, birth_thresh_0D)
        #longest_persistence_0D = np.max(life_0D)
        #mean_life_0D = np.mean(life_0D)
        #ph_entropy_0D = PH_entropy(ripser_diagram, 0)
        #Global_0D_features_AP_PE.append(np.concatenate(([mean_life_0D], [ph_entropy_0D])))
    
   
    
        #PH1 Features
        birth_thresh_1D = clean_arr(np.array(ripser_diagram[1][:,0]))
        death_thresh_1D = clean_arr(np.array(ripser_diagram[1][:,1]))
        life_1D = np.subtract(death_thresh_1D, birth_thresh_1D)
        longest_persistence_1D = np.max(life_1D)
        mean_life_1D = np.mean(life_1D)
        ph_entropy_1D = PH_entropy(ripser_diagram, 1)
        Global_1D_features_AP_PE.append(np.concatenate(([mean_life_1D], [ph_entropy_1D])))
    
       
                
        sub_Vertex_Importance = [0]*N #initialize with zeros for each subjects
        #print("Initialized Vertex Importance Profile =", sub_Vertex_Importance)
        
        
        #For each permutation of nodes compute the VIP
        
        
        Avg_sub_Vertex_Importance = np.array([0]*N, float)
        
        for j in range(0,N):
            #print("j=", j)
            sub_temp=sub
            sub_temp[[1, j],:] = sub_temp[[j, 1],:]
            sub_temp[:,[1, j]] = sub_temp[:,[j, 1]]
            np.fill_diagonal(sub_temp, 0)
            
            result = ripser(sub_temp, distance_matrix=True, do_cocycles=True)
            diagrams = result['dgms']
            D = result['dperm2all']
            cocycles = result['cocycles']
           
        
            #print(diagrams)
            #print(cocycles)
        
            #Visualization
            
            #dgm1 = diagrams[1]
            #idx = np.argmax(dgm1[:, 1] - dgm1[:, 0])
            #plot_diagrams(diagrams, show = False)
            #plt.scatter(dgm1[idx, 0], dgm1[idx, 1], 20, 'k', 'x')
            #plt.title("Max 1D birth = %.3g, death = %.3g"%(dgm1[idx, 0], dgm1[idx, 1]))
            #plt.show()
        

            
        
            #print("Number of 1-dim cocyles :", len(cocycles[1]))
            #print("1-dim cocyles :", cocycles[1])
            
            
            sub_Vertex_Importance = [0]*N
            for k,ar in enumerate(cocycles[1]):
            
                size_ar = len(ar)
                
                for rep_cyc in ar: 
                    for vertex in rep_cyc:
                    
                        sub_Vertex_Importance[vertex] = min(round(sub_Vertex_Importance[vertex]+round((1/size_ar), 5), 5), k+1)
            
            temp_vertex = sub_Vertex_Importance[j]
            sub_Vertex_Importance[j] = sub_Vertex_Importance[1]
            sub_Vertex_Importance[1] = temp_vertex
            
            #print(sub_Vertex_Importance)
            Avg_sub_Vertex_Importance += np.array(sub_Vertex_Importance)
            
        norm_arr = [len(cocycles[1])]*N # normalizing array
            
        Avg_sub_Vertex_Importance = np.subtract(Avg_sub_Vertex_Importance, norm_arr)
        Avg_sub_Vertex_Importance = Avg_sub_Vertex_Importance/np.max(Avg_sub_Vertex_Importance)
        Avg_sub_Vertex_Importance = np.array(Avg_sub_Vertex_Importance, dtype = np.float64)
        Avg_sub_Vertex_Importance = np.round_(Avg_sub_Vertex_Importance, decimals = 5)
        Avg_sub_Vertex_Importance = np.array(Avg_sub_Vertex_Importance, dtype = np.float64)
        Avg_sub_Vertex_Importance_NumCoCycles = np.concatenate((Avg_sub_Vertex_Importance, np.array([len(cocycles[1])])), axis=0)
        
        
        #adding subject ID 
        
        sub_id= subj_nets[i][0]
        
        Avg_Sub_ID_Vertex_Importance_NumCoCycles = np.concatenate((np.array([sub_id]), Avg_sub_Vertex_Importance_NumCoCycles), axis=0)
        
        
       
        #print("VIP=",Avg_sub_Vertex_Importance)
                 
        Vertex_Importance = np.vstack((Vertex_Importance, Avg_sub_Vertex_Importance ))
        Vertex_Importance_NumCoCyles = np.vstack((Vertex_Importance_NumCoCyles, Avg_sub_Vertex_Importance_NumCoCycles))
        Sub_ID_Vertex_Importance_NumCoCycles = np.vstack((Sub_ID_Vertex_Importance_NumCoCycles, Avg_Sub_ID_Vertex_Importance_NumCoCycles))
        
        
        
       
        
        
        #break;
    
    #return(Global_1D_features_AP_PE, Vertex_Importance_NumCoCyles)
    return(Global_1D_features_AP_PE, Sub_ID_Vertex_Importance_NumCoCycles)

In [73]:
Global_1D_features_AP_PE, Vertex_Importance_NumCoCyles = compute_cocycles(subj_mats)


100%|█████████████████████████████████████████| 410/410 [04:43<00:00,  1.45it/s]


In [6]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
#print(Global_1D_features_AP_PE)

In [7]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
#print(Vertex_Importance_NumCoCyles)

In [88]:
ADNI_fMRI_Global_Vertex_Importance_NumCoCyles = pd.DataFrame(Vertex_Importance_NumCoCyles)
ADNI_fMRI_Global_Vertex_Importance_NumCoCyles.to_csv("ADNI_fMRI_Global_Vertex_Importance_NumCoCyles.csv")

In [8]:
#print(ADNI_fMRI_Global_Vertex_Importance_NumCoCyles)

In [80]:
file_VIP = open("ADNI_fMRI_Global_Vertex_Importance_NumCoCyles_07312023.npy", 'rb')
np.save(file_VIP, np.array(Vertex_Importance_NumCoCyles))

In [79]:
file_AP_PE= open("ADNI_fMRI_Global_1D_features_AP_PE_07312023.npy", 'rb')
np.save(file_AP_PE, np.array(Global_1D_features_AP_PE))

In [2]:
VIP = np.load("ADNI_fMRI_Global_Vertex_Importance_NumCoCyles_07312023.npy")

In [3]:
demo = pd.read_csv('subject_demographics.csv')

In [10]:
sub_ids = VIP[:,0]
n = len(sub_ids)
sub_ids = sub_ids[1: n]
print(len(sub_ids))

410


In [3]:
#display(demo)

In [21]:
filtered_demo = demo[demo["PTID"].isin(sub_ids)]

In [4]:
#print(filtered_demo)

In [26]:
subjects = filtered_demo[['PTID', 'DX_bl', 'PTGENDER']].groupby(['DX_bl', 'PTGENDER'])
print(subjects.count())

                PTID
DX_bl PTGENDER      
AD    Female       8
      Male        14
CN    Female      58
      Male        56
EMCI  Female      51
      Male        60
LMCI  Female      38
      Male        48
SMC   Female      45
      Male        32


In [30]:
demo_age = filtered_demo[['PTID', 'DX_bl', 'AGE']]

In [5]:
#display(demo_age)

In [33]:
output = demo_age.groupby(['DX_bl'], as_index=False).agg({'AGE':['mean','std']})
display(output)

In [36]:
demo_age_edu = filtered_demo[['PTID', 'DX_bl', 'AGE', 'PTEDUCAT']]

In [37]:
output = demo_age_edu.groupby(['DX_bl'], as_index=False).agg({'PTEDUCAT':['mean','std']})
display(output)

DX_bl   PTEDUCAT          
              mean       std
0    AD  15.772727  2.860766
1    CN  16.421053  2.555212
2  EMCI  16.378378  2.504738
3  LMCI  16.104651  2.731210
4   SMC  16.402597  2.466938